# YOLOv8n Implementação
Este notebook demonstra como usar o YOLOv8n para tarefas de detecção de objetos. A seguir, percorremos as etapas principais, incluindo o carregamento de dados, o treinamento do modelo e a avaliação de seu desempenho.

## Importando bibliotecas

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image
import xml.etree.ElementTree as ET
from ultralytics import YOLO

## Configurar diretórios
### Nesta seção, configuramos a estrutura de diretórios para organizar os dados:

- source_dataset_path contém os dados originais com anotações em XML.
- target_dataset_path será a pasta organizada, com subpastas para imagens e rótulos, pronta para ser usada em treinamentos e inferência com o modelo YOLO.

In [ ]:
source_dataset_path = 'dataset_xml_format'
target_dataset_path = 'dataset'
image_path = f"{target_dataset_path}/images"
label_path = f"{target_dataset_path}/labels"

In [3]:
os.makedirs(f"{image_path}/train", exist_ok=True)
os.makedirs(f"{image_path}/val", exist_ok=True)
os.makedirs(f"{image_path}/test", exist_ok=True)
os.makedirs(f"{label_path}/train", exist_ok=True)
os.makedirs(f"{label_path}/val", exist_ok=True)
os.makedirs(f"{label_path}/test", exist_ok=True)

## Separando arquivos de imagens e anotações (arquivos XML que contêm bounding boxes).

In [4]:
images = [f for f in os.listdir(source_dataset_path) if f.endswith('.png') or f.endswith('.jpg')]
xmls = [f for f in os.listdir(source_dataset_path) if f.endswith('.xml')]

## Dividindo a base em Treino, Teste e Validação

In [5]:
train_images, temp_images = train_test_split(images, test_size=0.3, random_state=42)
val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

## Verificando a quantidade de imagens em cada classe

In [6]:
print(f"Train images: {len(train_images)}")
print(f"Val images: {len(val_images)}")
print(f"Test images: {len(test_images)}")

Train images: 760
Val images: 163
Test images: 164


## Convertendo um arquivo de anotação XML para o formato TXT.

In [7]:
def safe_move(src, dest):
    if not os.path.exists(dest):
        shutil.copy(src, dest)

In [ ]:
def convert_to_txt(xml_file, txt_file, img_width, img_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    with open(txt_file, 'w') as f:
        for obj in root.findall('object'):
            name = obj.find('name').text
            class_id = 0 
            xmlbox = obj.find('bndbox')
            xmin = int(xmlbox.find('xmin').text)
            ymin = int(xmlbox.find('ymin').text)
            xmax = int(xmlbox.find('xmax').text)
            ymax = int(xmlbox.find('ymax').text)

            x_center = ((xmin + xmax) / 2) / img_width
            y_center = ((ymin + ymax) / 2) / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1):
                print(f"Coordenadas fora dos limites em {xml_file}")
                continue

            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

## Organizando e convertendo dados de imagens e anotações para serem usados em treinamento, validação e teste em um modelo YOLO.

In [ ]:
for img in images:
    if img in train_images:
        split = 'train'
    elif img in val_images:
        split = 'val'
    else:
        split = 'test'
    
    img_path = os.path.join(source_dataset_path, img)
    xml_path = os.path.join(source_dataset_path, img.replace('.png', '.xml').replace('.jpg', '.xml'))

    safe_move(img_path, f"{image_path}/{split}/{img}")
    if os.path.exists(xml_path):
        img_obj = Image.open(img_path)
        img_width, img_height = img_obj.size
        txt_file = os.path.join(source_dataset_path, img.replace('.png', '.txt').replace('.jpg', '.txt'))
        convert_to_txt(xml_path, txt_file, img_width, img_height)
        safe_move(txt_file, f"{label_path}/{split}/{os.path.basename(txt_file)}")

## Configurando e salvando um arquivo YAML para treinar um modelo YOLO.

In [10]:
yaml_content = """
path: dataset
train: images/train
val: images/val
test: images/test
names:
  0: drone
"""

In [11]:
with open('config.yaml', 'w') as f:
    f.write(yaml_content)

## Definindo a quantidade de épocas que o modelo vai ter

In [12]:
epochs = 50

## Treinando o modelo

In [ ]:
model = YOLO('yolov8n.pt') 
model.train(data='config.yaml', epochs=epochs, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.32 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.31 🚀 Python-3.12.7 torch-2.5.1+cu124 CPU (Intel Core(TM) i3-7020U 2.30GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=config.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False,

train: Scanning /home/avivyh/Documentos/SINFO/dataset/labels/train.cache... 760 images, 0 backgrounds, 0 corrupt: 100%|██████████| 760/760 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/avivyh/Documentos/SINFO/dataset/images/train/pic_722.jpg: corrupt JPEG restored and saved



val: Scanning /home/avivyh/Documentos/SINFO/dataset/labels/val.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|██████████| 163/163 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        163        180      0.979      0.961       0.99      0.704



50 epochs completed in 7.135 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.31 🚀 Python-3.12.7 torch-2.5.1+cu124 CPU (Intel Core(TM) i3-7020U 2.30GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 3/6 [00:13<00:13,  4.50s/it]libpng warning: iCCP: known incorrect sRGB profile
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:26<00:00,  4.36s/it]


                   all        163        180      0.969      0.961      0.988      0.706
Speed: 2.3ms preprocess, 134.5ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e1d4b993ce0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

## Validando e testando o modelo

In [ ]:
metrics_val = model.val(split='val')
print("Métricas de Validação:", metrics_val)

metrics_test = model.val(split='test')
print("Métricas de Teste:", metrics_test)

Ultralytics 8.3.31 🚀 Python-3.12.7 torch-2.5.1+cu124 CPU (Intel Core(TM) i3-7020U 2.30GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/avivyh/Documentos/SINFO/dataset/labels/val.cache... 163 images, 0 backgrounds, 0 corrupt: 100%|██████████| 163/163 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  64%|██████▎   | 7/11 [00:15<00:09,  2.31s/it]libpng warning: iCCP: known incorrect sRGB profile
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:25<00:00,  2.28s/it]


                   all        163        180      0.971      0.961      0.989      0.709
Speed: 2.2ms preprocess, 129.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train2
Métricas de Validação: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e1d2c99c890>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.0310

val: Scanning /home/avivyh/Documentos/SINFO/dataset/labels/test... 164 images, 0 backgrounds, 0 corrupt: 100%|██████████| 164/164 [00:00<00:00, 401.97it/s]

val: New cache created: /home/avivyh/Documentos/SINFO/dataset/labels/test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  45%|████▌     | 5/11 [00:10<00:12,  2.11s/it]libpng warning: iCCP: known incorrect sRGB profile
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:25<00:00,  2.30s/it]


                   all        164        176      0.951      0.949      0.958      0.659
Speed: 2.2ms preprocess, 129.2ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train3
Métricas de Teste: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e1d49fb1d00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031, 